In [ ]:
%cd ../code_

In [ ]:
from tableTestingMicroScript import *
import pandas as pd
import os
import ast
from utils import num_tokens_from_string

In [ ]:
from prompts import EXAMPLES, EXAMPLE_Dictionary
import pandas as pd
from utils import num_tokens_from_string


def gather_Examples_Prompt(tab_format, test_case):

    TF, TC = tab_format.__class__.__name__, test_case
    Examples = EXAMPLES.replace("[Data_format_example1]", EXAMPLE_Dictionary[f"TF_EX1_{TF}"])\
        .replace("[QA1]", EXAMPLE_Dictionary[f"Ex1_QA1_{TC}"])\
        .replace("[Data_format_example2]", EXAMPLE_Dictionary[f"TF_EX1_{TF}"])\
        .replace("[QA2]", EXAMPLE_Dictionary[f"Ex1_QA2_{TC}"])
    return Examples


def get_max_no_of_rows_feedable(df, FormatFunction, Micro=True):

    per_test = []
    test_case = ["NavigationTests", "ColumnLookupTests",
                 "RowLookupTests", "DataTypeLookupTests"]

    for tt in test_case:
        exp = gather_Examples_Prompt(FormatFunction, tt)

        for i in range(min(df.shape[0], 100), 2, -1):
            if Micro:
                total_token = 30 + num_tokens_from_string(exp)+11+num_tokens_from_string(
                    str(FormatFunction.formatting(df.head(i))))+100
            else:
                total_token = 100 + num_tokens_from_string(exp)+11+num_tokens_from_string(
                    str(FormatFunction.formatting(df.head(i))))*2.5

            if total_token < 4081:

                per_test.append(i)
                break
    return (min(per_test))

In [ ]:

benchmark_data_path = "../BenchmarkDatasetClean"
benchmark_dataset = [f for f in os.listdir(
    benchmark_data_path) if f.endswith(".csv")]
benchmark_save_path = "../BenchmarkResults_Micro_tests/"
if not os.path.exists(benchmark_save_path):
    os.makedirs(benchmark_save_path)
open_api_key = os.env("OPENAI_API_KEY")
for bd in benchmark_dataset:
    print(bd)
    df = pd.read_csv(os.path.join(benchmark_data_path, bd))
    save_path = os.path.join(benchmark_save_path, bd)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    table_formats = [HTMLFormat(), HTMLNoSpaceFormat(), MarkdownFormat(), JsonFormat(
    ), DataMatrixFormat(), DFloaderFormat(), TabSeparatedFormat(), CommaSeparatedFormat()],
    max_rows = min([get_max_no_of_rows_feedable(df, format)
                   for format in table_formats])

    df = df.head(max_rows)
    print(df.shape)
    suite = TableExperimentSuite(LLMTableLearner(),
                                 [HTMLFormat(), HTMLNoSpaceFormat(), MarkdownFormat(), JsonFormat(), DataMatrixFormat(
                                 ), DFloaderFormat(), TabSeparatedFormat(), CommaSeparatedFormat()],
                                 [ArbitraryColumnNames(),
                                  ShuffleRows(),
                                  TransposeTable(),
                                  ColumnCluster(),
                                  SerializeTable(),
                                  ShuffleColumns(),
                                  ShuffleColumnNames(),
                                  OriginalData(),
                                  SequentialColumnNames()
                                  ],
                                 [NavigationTests(), ColumnLookupTests(),
                                  RowLookupTests(), DataTypeLookupTests()],
                                 save_path, open_api_key)
    output = suite.run_experiment(df, per_table_op=10, per_test_gen=10)
    col_names = ["temperature", "tableFormat", "TableManipulation", "TestCase", "TestQuestion", "TestExpectedAnswer", "TestCaseType",
                 "pass_1", "pass_3", "pass_5", "pass_10", "pass_15", "TestCaseTuple", "Answers_top_15", "Answer_top_1", "Result", "Error"]
    output_df = pd.DataFrame(output, columns=col_names)
    output_df.to_csv(os.path.join(
        save_path, f"Micro_{bd}_output.csv"), index=None)
    try:
        output_df.to_pickle(os.path.join(
            save_path, f"Micro_{bd}_output.pkl"), index=None)
    except:
        pass